# Préparation et nettoyage des données DVF

**Objectif** : préparer un jeu de données propre pour l'analyse des **locaux commerciaux / boutiques** à **Paris** (arrondissements, prix/m², année, géolocalisation).



In [62]:
import pandas as pd
import numpy as np
import os

In [63]:
df = pd.read_csv('dvf.csv')

C:\Users\victo\AppData\Local\Temp\ipykernel_13904\2023539279.py:1: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dvf.csv')


##### on filtre les colonnes importantes

In [64]:
colonnes_importantes = [
    'date_mutation',
    'valeur_fonciere',
    'code_postal',
    'type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'longitude',
    'latitude',
    'adresse_nom_voie',
    'nom_commune'
]

df = df[colonnes_importantes]

##### On enlève les lignes qui contiennent des valeurs NaN

In [65]:
df = df.dropna(subset=['valeur_fonciere', 'surface_reelle_bati', 
                       'nombre_pieces_principales',
                       'latitude', 'longitude'])

In [66]:
df.isna().sum()

date_mutation                 0
valeur_fonciere               0
code_postal                  79
type_local                    0
surface_reelle_bati           0
nombre_pieces_principales     0
longitude                     0
latitude                      0
adresse_nom_voie              4
nom_commune                   0
dtype: int64

##### On modifie le typage data en datetime

In [67]:
df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')

##### On ne retient que le type : "Local industriel. commercial ou assimilé"

In [68]:
type_autorise = ['Local industriel. commercial ou assimilé']
mask_type = df['type_local'].isin(type_autorise)
df = df[mask_type]

##### Puis on retire du dataframe la colonne "type_local"

In [69]:
df = df.drop('type_local', axis=1)

##### On s'intéresse uniquement au département de Paris

In [70]:
mask_paris = df['code_postal'].astype(str).str.startswith('75')
df = df[mask_paris]

##### Création d'une feature prix m²

In [71]:
df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']

##### On supprime les valeurs aberrantes au m²

In [72]:
df = df[(df['prix_m2'] > 500) & (df['prix_m2'] < 50000)]

##### Extraction de l'arrondissement (normalisation de code_postal)

In [73]:
df['code_postal'] = df['code_postal'].astype(int).astype(str).str.zfill(5)
df['arrondissement'] = pd.to_numeric(df['code_postal'].str[-2:], errors='coerce')
df.loc[~df['arrondissement'].between(1,20), 'arrondissement'] = np.nan
df['arrondissement'] = df['arrondissement'].dropna().astype(int)

##### on supprime la colonne code_postal (c'est toujours Paris)

In [74]:
df = df.drop('code_postal', axis=1)

##### Certains nom de communes ne sont pas Paris. On les supprime

In [75]:
df = df[df['nom_commune'].str.startswith('Paris')]

# Nettoyage BDD Loyer

In [76]:
loyer_df = pd.read_csv("loyers.csv", sep=';', encoding='utf-8')

##### On supprime les colonnes inutiles

In [77]:
colonnes_a_supprimer = ["Secteurs géographiques", "Numéro du quartier", "Ville","geo_shape"]
loyer_df = loyer_df.drop(columns=colonnes_a_supprimer, errors='ignore')

##### Le dataframe ne contient pas de valeurs NaN ou null, ni de doublons

##### Nettoyage des coordonnées GPS (format "lat, lon")

In [78]:
loyer_df[['lat', 'lon']] = loyer_df['geo_point_2d'].str.split(',', expand=True)
loyer_df['lat'] = loyer_df['lat'].astype(float)
loyer_df['lon'] = loyer_df['lon'].astype(float)

suppression de la colonne 'geo_point_2d'

In [79]:
loyer_df = loyer_df.drop('geo_point_2d', axis=1)

In [80]:
df.head()

,date_mutation,valeur_fonciere,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,adresse_nom_voie,nom_commune,prix_m2,arrondissement
3384378,2024-01-08,170000.0,14.0,0.0,2.351308,48.866263,PAS BASFOUR,Paris 2e Arrondissement,12142.857143,2.0
3384389,2024-01-04,650000.0,59.0,0.0,2.330483,48.874388,RUE JOUBERT,Paris 9e Arrondissement,11016.949153,9.0
3384458,2024-01-10,500000.0,51.0,0.0,2.354192,48.888191,RUE DOUDEAUVILLE,Paris 18e Arrondissement,9803.921569,18.0
3384464,2024-01-10,378000.0,60.0,0.0,2.370435,48.870213,RUE DU FBG DU TEMPLE,Paris 10e Arrondissement,6300.000000,10.0
3384465,2024-01-10,378000.0,46.0,0.0,2.370435,48.870213,RUE DU FBG DU TEMPLE,Paris 10e Arrondissement,8217.391304,10.0


# Jointure du dataframe des valeurs foncières et de celui des loyers

In [81]:
from scipy.spatial import cKDTree

In [82]:
coords_quartiers = loyer_df[['lat', 'lon']].values
tree = cKDTree(coords_quartiers)

In [83]:
coords_dvf = df[['latitude', 'longitude']].values

In [84]:
distances, indices = tree.query(coords_dvf)

Ajouter les colonnes au DataFrame DVF

In [85]:
df['quartier'] = loyer_df.iloc[indices]['Nom du quartier'].values
df['numero_insee_quartier'] = loyer_df.iloc[indices]['Numéro INSEE du quartier'].values
df['loyer_ref_m2'] = loyer_df.iloc[indices]['Loyers de référence'].values
df['loyer_majore_m2'] = loyer_df.iloc[indices]['Loyers de référence majorés'].values
df['loyer_minore_m2'] = loyer_df.iloc[indices]['Loyers de référence minorés'].values
df['distance_quartier_m'] = distances * 111000  # Conversion approximative en mètres

checking s'il y a pas des lignes ou le quartier est loin de la réalité

In [86]:
df = df[df['distance_quartier_m'] <= 1000]

In [87]:
df.head()

,date_mutation,valeur_fonciere,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,adresse_nom_voie,nom_commune,prix_m2,arrondissement,quartier,numero_insee_quartier,loyer_ref_m2,loyer_majore_m2,loyer_minore_m2,distance_quartier_m
3384378,2024-01-08,170000.0,14.0,0.0,2.351308,48.866263,PAS BASFOUR,Paris 2e Arrondissement,12142.857143,2.0,Bonne-Nouvelle,7510208,23.0,27.60,16.10,168.138266
3384389,2024-01-04,650000.0,59.0,0.0,2.330483,48.874388,RUE JOUBERT,Paris 9e Arrondissement,11016.949153,9.0,Chaussée-d'Antin,7510934,24.3,29.20,17.00,219.094129
3384458,2024-01-10,500000.0,51.0,0.0,2.354192,48.888191,RUE DOUDEAUVILLE,Paris 18e Arrondissement,9803.921569,18.0,Goutte-d'Or,7511871,20.4,24.48,14.28,462.845576
3384464,2024-01-10,378000.0,60.0,0.0,2.370435,48.870213,RUE DU FBG DU TEMPLE,Paris 10e Arrondissement,6300.000000,10.0,Folie-Méricourt,7511141,27.4,32.90,19.20,419.677569
3384465,2024-01-10,378000.0,46.0,0.0,2.370435,48.870213,RUE DU FBG DU TEMPLE,Paris 10e Arrondissement,8217.391304,10.0,Folie-Méricourt,7511141,27.4,32.90,19.20,419.677569


In [89]:
df['nombre_pieces_principales'].value_counts()

nombre_pieces_principales
0.0    2913
Name: count, dtype: int64

In [90]:
df.to_csv("dvf_loyers.csv", sep=";", encoding="utf-8", index=False)